In [34]:
import tensorflow as tf
import numpy as np
import timeit
from sklearn.utils import shuffle
from load_cifar import one_hot_encoding
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

<h1>Extract MNIST data</h1>

In [35]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding, reshape = False (that means images are not flatten)
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [36]:
x_train, y_train           = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test             = mnist.test.images, mnist.test.labels

#pad images with 0s (28x28 to 32x32)
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_validation = np.pad(x_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

print(x_train.shape)

(55000, 32, 32, 1)


<h1>Define hyperparameter</h1>

In [43]:
epochs = 10
batch_size = 128
lr = 0.01

In [44]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [45]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.float32, (None))

<h1>Define LeNet-5</h1>

In [46]:
def lenet5(x):
    
    ###### Convolution layer 6 @ 28x28 ######

    W1 = tf.Variable(tf.truncated_normal(shape=(5,5,1,6), mean = 0, stddev = 0.1))
    b1 = tf.Variable(tf.zeros(6))

    l1 = tf.nn.conv2d(x, W1, strides=[1,1,1,1], padding='VALID') + b1
    l1 = tf.nn.relu(l1)
    l1 = tf.nn.max_pool(l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

    ###### Convolution layer 16 @ 10x10 ######

    W2 = tf.Variable(tf.truncated_normal(shape=(5,5,6,16), mean = 0, stddev = 0.1))
    b1 = tf.Variable(tf.zeros(16))

    l2 = tf.nn.conv2d(l1, W2, strides=[1,1,1,1], padding='VALID')
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.max_pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

    ###### Fully connected (400 --> 120) ######
    flat = tf.contrib.layers.flatten(l2)

    W3 = tf.Variable(tf.truncated_normal(shape=(400,120), mean = 0, stddev = 0.1))
    b3 = tf.Variable(tf.zeros(120))

    l3 = tf.matmul(flat, W3) + b3
    l3 = tf.nn.relu(l3)

    ###### Fully connected (120 --> 84) ######
    W4 = tf.Variable(tf.truncated_normal(shape=(120,84), mean = 0, stddev = 0.1))
    b4 = tf.Variable(tf.zeros(84))

    l4 = tf.matmul(l3, W4) + b4
    l4 = tf.nn.relu(l4)

    ###### Fully connected (84 --> 10) ######
    W5 = tf.Variable(tf.truncated_normal(shape=(84,10), mean = 0, stddev = 0.1))
    b5 = tf.Variable(tf.zeros(10))
    
    logits = tf.matmul(l4, W5) + b5
    
    return logits 

<h1>Cost and optimization</h1>

In [47]:
logits = lenet5(x)

#define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y),name='loss')

#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost)

corr_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(corr_pred, tf.float32))


def get_stats(session, cost, accuracy, b_feat, b_labels, v_feat, v_labels):
    loss = sess.run(cost,
                   feed_dict={
                       x: b_feat,
                       y: b_labels, 
                   })
    
    acc = sess.run(accuracy, 
                  feed_dict={
                      x: v_feat,
                      y: v_labels,
                  })
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, acc))

<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [48]:
#Initialize the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(epochs):
        
        #fetch batch
        for j in range(0, len(x_train), batch_size):
            k = j + batch_size
            batch_x, batch_y = x_train[j:k], y_train[j:k]
            sess.run(train_op, feed_dict={x:batch_x, y:batch_y})
        
        print('Epoch {:>2}, MNIST Batch {}:  '.format(i + 1, i), end='')
        get_stats(sess, cost, acc, batch_x, batch_y, x_validation, y_validation)

Epoch  1, MNIST Batch 0:  Loss:     0.5409 Validation Accuracy: 0.817400
Epoch  2, MNIST Batch 1:  Loss:     0.2929 Validation Accuracy: 0.900600
Epoch  3, MNIST Batch 2:  Loss:     0.2582 Validation Accuracy: 0.930600
Epoch  4, MNIST Batch 3:  Loss:     0.2489 Validation Accuracy: 0.945000
Epoch  5, MNIST Batch 4:  Loss:     0.2487 Validation Accuracy: 0.952800
Epoch  6, MNIST Batch 5:  Loss:     0.2501 Validation Accuracy: 0.960200
Epoch  7, MNIST Batch 6:  Loss:     0.2520 Validation Accuracy: 0.964600
Epoch  8, MNIST Batch 7:  Loss:     0.2508 Validation Accuracy: 0.968600
Epoch  9, MNIST Batch 8:  Loss:     0.2488 Validation Accuracy: 0.971000
Epoch 10, MNIST Batch 9:  Loss:     0.2472 Validation Accuracy: 0.972400
